In [ ]:
green_areas =r'D:/osm2lulc01/osm/osmgareas.shp'

green_fid = 'ffid'

dpop = r'D:\osm2lulc01\guavsforest.gdb\pop\bgri21'

popcol = 'N_INDIVIDUOS'

outfeat = r'D:\osm2lulc01\guavsforest.gdb\osm\result_pop'

tmpws = r'D:\osm2lulc01\guavsforest.gdb\corre'

In [ ]:
import arcpy
import os

from gesri.dp import copy_feat
from glass.pys.oss import fprop
from gesri.rd.shp import shp_to_lyr
from gesri.tbl.attr import geom_attr_to_shp
from gesri.tbl.col import add_col, cols_calc
from gesri.gp.ovl import intersection
from gesri.gp.gen import dissolve
from gesri.tbl.joins import join_table

In [ ]:
srs = "PROJCS[\"ETRS_1989_Portugal_TM06\",GEOGCS[\"GCS_ETRS_1989\",DATUM[\"D_ETRS_1989\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-8.133108333333334],PARAMETER[\"Scale_Factor\",1.0],PARAMETER[\"Latitude_Of_Origin\",39.66825833333333],UNIT[\"Meter\",1.0]]"

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
# Get population density

popshp = copy_feat(dpop, os.path.join(
    tmpws, 'popdata'
))

poplyr = shp_to_lyr(popshp)

poplyr = geom_attr_to_shp(
    poplyr, "areakm",
    geom_attr="AREA", srs=srs,
    area_unit="SQUARE_KILOMETERS"
)

poplyr = add_col(poplyr, "dpop", "DOUBLE", None)

poplyr = cols_calc(poplyr, "dpop", f"!{popcol}!/!areakm!")

In [ ]:
# Intersect population and green areas

glyr = shp_to_lyr(green_areas)

ointersect = os.path.join(tmpws, 'i_pop_green')
ilyr = intersection([glyr, poplyr], ointersect)

In [ ]:
# Dissolve

odiss = os.path.join(tmpws, 'pop_green_diss')
disslyr = dissolve(
    ilyr, odiss, green_fid,
    statistics=[["dpop", "MEAN"]]
)

In [ ]:
# Join 
cp_green = copy_feat(green_areas, outfeat)

green_lyr = join_table(
    cp_green, disslyr,
    green_fid, green_fid,
    cols=["MEAN_dpop"]
)